# Context-Aware RAG using AIQ (Alpha)

This notebook demonstrates how to use NVIDIA's Context-Aware RAG (CA-RAG) system with AIQ for document processing and question answering. The example shows how to:
- Ingest documents using AIQ
- Perform question answering using AIQ

## Prerequisites

1. NVIDIA API Keys:
   - Get your API keys from: https://build.nvidia.com
   - Export the following environment variables:
     ```bash
     export NVIDIA_API_KEY=your_api_key
     ```

2. Install AIQ and Context-Aware RAG:
   - Follow the installation instructions in the documentation
   - Make sure you have both AIQ and Context-Aware RAG installed in your environment

3. Set up Vector Database (Milvus):
   - Install and start Milvus
   - Export environment variables:
     ```bash
     export MILVUS_HOST=localhost
     export MILVUS_PORT=19530
     ```

In [ ]:
%env NVIDIA_API_KEY=<NVIDIA_API_KEY>
%env MILVUS_HOST=<MILVUS_HOST>
%env MILVUS_PORT=<MILVUS_PORT>

In [2]:
import os
from pathlib import Path
import json

# Verify environment variables
assert os.getenv('NVIDIA_API_KEY'), "Please set NVIDIA_API_KEY environment variable"
assert os.getenv('MILVUS_HOST'), "Please set MILVUS_HOST environment variable"
assert os.getenv('MILVUS_PORT'), "Please set MILVUS_PORT environment variable"

## Start AIQ Services

We need to start two AIQ services:
1. Ingestion service - for processing and ingesting PDF documents
2. Retrieval service - for answering questions based on ingested documents

### Starting AIQ Services

To start the AIQ services, run the following commands in separate terminal windows:

### 1. Start Ingestion Service

Terminal 1:

```bash
aiq serve --config_file=./src/vss_ctx_rag/aiq_config/workflow/config-ingestion-workflow.yml --port 8000
```

### 2. Start Retrieval Service

Terminal 2:
```bash
aiq serve --config_file=./src/vss_ctx_rag/aiq_config/workflow/config-retrieval-workflow.yml --port 8001
```

## Ingest Documents

In [3]:
# Sample Documents

add_doc_data_list = [
    "User1: Hi how are you?",
    "User2: I am good. How are you?",
    "User1: I am great too. Thanks for asking",
    "User2: So what did you do over the weekend?",
    "User1: I went hiking to Mission Peak",
    "User3: Guys there is a fire. Let us get out of here"
]


In [4]:
import requests

def ingest_document(doc_list):
    """Ingest document content using the AIQ ingestion service."""
    # Ingest documents
    ingestion_url = "http://localhost:8000/generate"
    headers = {"Content-Type": "application/json"}
    try:
        for add_doc_data in doc_list:
            print(f"Adding document {add_doc_data}")
            data = {
                "text": add_doc_data,
            }
            response = requests.post(
                ingestion_url, headers=headers, data=json.dumps(data)
            )
            response.raise_for_status()  # Raise an error for bad responses
            print(f"Added document {response.json()}")
        return {"success": "All documents uploaded successfully"}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

In [ ]:
ingestion_result = ingest_document(add_doc_data_list)
print(f"Ingestion result: {ingestion_result}")

## Ask Questions

Now that we've ingested the documents, let's ask some questions using the retrieval service.

In [ ]:
def ask_question(question):
    """Ask a question using the AIQ retrieval service."""
    url = "http://localhost:8001/generate"
    headers = {"Content-Type": "application/json"}
    data = {
        "text": question,
    }
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()

# Example questions
questions = [
    "summarize the current conversation",
]

for question in questions:
    print(f"\nQuestion: {question}")
    answer = ask_question(question)
    print(f"Answer: {answer}")

## Cleanup

When you're done, make sure to stop the AIQ services.
When you're done, make sure to stop the AIQ services.

To stop the service running in Terminal 1 and Terminal 2: Press `Ctrl+C` in the terminal windows.
